# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("disaster_messages", con=engine)

In [3]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26210,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26211,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df['message']
Y = df.iloc[:, 4:]

In [5]:
Y.head(1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    """
    This function will tokenize texts.
    """
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)

In [11]:
def test_model(y_test, y_pred):
    
    """
    This function will test the model and produce classification report.
    """
    for index, column in enumerate(y_test):
        print(column, classification_report(y_test[column], y_pred[:, index]))

In [12]:
test_model(y_test, y_pred)

related               precision    recall  f1-score   support

           0       0.75      0.27      0.40      1555
           1       0.81      0.97      0.88      4999

    accuracy                           0.81      6554
   macro avg       0.78      0.62      0.64      6554
weighted avg       0.80      0.81      0.77      6554

request               precision    recall  f1-score   support

           0       0.90      0.99      0.94      5461
           1       0.88      0.43      0.58      1093

    accuracy                           0.90      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.90      0.88      6554

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      655

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

           0       0.93      0.99      0.96      5832
           1       0.85      0.37      0.51       722

    accuracy                           0.92      6554
   macro avg       0.89      0.68      0.74      6554
weighted avg       0.92      0.92      0.91      6554

shelter               precision    recall  f1-score   support

           0       0.93      1.00      0.96      5974
           1       0.86      0.22      0.34       580

    accuracy                           0.93      6554
   macro avg       0.89      0.61      0.65      6554
weighted avg       0.92      0.93      0.91      6554

clothing               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6450
           1       0.56      0.09      0.15       104

    accuracy                           0.98      6554
   macro avg       0.77      0.54      0.57      6554
weighted avg       0.98      0.98      0.98      6554

m

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x0000015B4F6D9E58>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          ccp_alpha=0.0,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_dept

In [14]:
# specify parameters for grid search 
parameters = {
    'clf__estimator__n_estimators' : [50, 100]
}

In [15]:
# create grid search object 
cv = GridSearchCV(pipeline, param_grid=parameters)

cv

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [16]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [17]:
cv.best_params_

{'clf__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
y_pred = cv.predict(X_test)

In [19]:
test_model(y_test, y_pred)

related               precision    recall  f1-score   support

           0       0.77      0.27      0.40      1555
           1       0.81      0.97      0.89      4999

    accuracy                           0.81      6554
   macro avg       0.79      0.62      0.64      6554
weighted avg       0.80      0.81      0.77      6554

request               precision    recall  f1-score   support

           0       0.89      0.99      0.94      5461
           1       0.87      0.40      0.55      1093

    accuracy                           0.89      6554
   macro avg       0.88      0.70      0.74      6554
weighted avg       0.89      0.89      0.87      6554

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6526
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      655

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6143
           1       0.90      0.24      0.37       411

    accuracy                           0.95      6554
   macro avg       0.92      0.62      0.67      6554
weighted avg       0.95      0.95      0.94      6554

food               precision    recall  f1-score   support

           0       0.93      0.99      0.96      5832
           1       0.86      0.37      0.52       722

    accuracy                           0.92      6554
   macro avg       0.89      0.68      0.74      6554
weighted avg       0.92      0.92      0.91      6554

shelter               precision    recall  f1-score   support

           0       0.93      1.00      0.96      5974
           1       0.85      0.21      0.33       580

    accuracy                           0.93      6554
   macro avg       0.89      0.60      0.65      6554
weighted avg       0.92      0.93      0.91      6554

cloth

In [20]:
accuracy = (y_pred == y_test).mean()
accuracy

related                   0.807904
request                   0.890601
offer                     0.995728
aid_related               0.763045
medical_help              0.923253
medical_products          0.951327
search_and_rescue         0.972078
security                  0.980317
military                  0.968721
child_alone               1.000000
water                     0.950412
food                      0.924016
shelter                   0.926457
clothing                  0.984284
money                     0.978334
missing_people            0.989167
refugees                  0.969942
death                     0.961398
other_aid                 0.871224
infrastructure_related    0.930882
transport                 0.953158
buildings                 0.952090
electricity               0.977418
tools                     0.993897
hospitals                 0.989167
shops                     0.994202
aid_centers               0.985658
other_infrastructure      0.954074
weather_related     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
import pickle
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.